In [1]:
import mysql.connector
from mysql.connector import errorcode

# Connect to MySQL server (not the Fintech DB yet)
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Msis@123'  # Replace with your root password
)
cursor = conn.cursor()

# Drop and create database, user, and grant privileges
commands = [
    "DROP DATABASE IF EXISTS Fintechs;",
    "CREATE DATABASE Fintechs;",
    "DROP USER IF EXISTS 'Mdbd'@'localhost';",
    "CREATE USER 'Mdbd'@'localhost' IDENTIFIED BY 'Mdbd@123';",
    "GRANT ALL PRIVILEGES ON Fintechs.* TO 'Mdbd'@'localhost';",
    "FLUSH PRIVILEGES;"
]

for cmd in commands:
    cursor.execute(cmd)

cursor.close()
conn.close()
print("Database and user created.")

Database and user created.


In [6]:
# Reconnect using new DB and user
conn = mysql.connector.connect(
    host='localhost',
    user='Mdbd',
    password='Mdbd@123',
    database='Fintechs'
)
cursor = conn.cursor()

table_sql = """
CREATE TABLE Users (
    user_id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL UNIQUE,
    date_of_birth DATE NOT NULL,
    registration_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    INDEX idx_email (email)
);

CREATE TABLE Accounts (
    account_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    account_type ENUM('savings', 'checking', 'brokerage') NOT NULL,
    balance DECIMAL(15, 2) NOT NULL DEFAULT 0.00,
    currency VARCHAR(3) NOT NULL DEFAULT 'INR',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    INDEX idx_user_id (user_id),
    INDEX idx_account_type (account_type)
);

CREATE TABLE Transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    account_id INT NOT NULL,
    amount DECIMAL(15, 2) NOT NULL,
    transaction_type ENUM('credit', 'debit') NOT NULL,
    transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    description VARCHAR(255),
    FOREIGN KEY (account_id) REFERENCES Accounts(account_id),
    INDEX idx_account_id (account_id),
    INDEX idx_transaction_date (transaction_date)
);

CREATE TABLE Investments (
    investment_id INT AUTO_INCREMENT PRIMARY KEY,
    account_id INT NOT NULL,
    investment_type ENUM('stocks', 'bonds', 'mutual_funds', 'crypto') NOT NULL,
    amount DECIMAL(15, 2) NOT NULL,
    purchase_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    current_value DECIMAL(15, 2),
    FOREIGN KEY (account_id) REFERENCES Accounts(account_id),
    INDEX idx_account_id (account_id),
    INDEX idx_investment_type (investment_type)
);

CREATE TABLE Loans (
    loan_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    loan_amount DECIMAL(15, 2) NOT NULL,
    interest_rate DECIMAL(5, 2) NOT NULL,
    loan_term_months INT NOT NULL,
    loan_start_date DATE NOT NULL,
    loan_status ENUM('active', 'closed', 'defaulted') NOT NULL DEFAULT 'active',
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    INDEX idx_user_id (user_id),
    INDEX idx_loan_status (loan_status)
);
"""

for stmt in table_sql.strip().split(';'):
    if stmt.strip():
        cursor.execute(stmt + ';')

conn.commit()
cursor.close()
print("Tables created.")


Tables created.


In [7]:
cursor = conn.cursor()

# Insert data into Users
cursor.execute("""
INSERT INTO Users (first_name, last_name, email, date_of_birth) VALUES
('Sanjay', 'Reddy', 'sanjay.reddy@example.com', '2001-09-03'),
('Chandana', 'Ramanahali', 'chandana.ramanahali@example.com', '2002-07-10'),
('Sakshi', 'Mayya', 'sakshi.mayya@example.com', '2000-11-03'),
('Bhoomika', 'Hegde', 'bhoomika.hegde@example.com', '2001-10-02'),
('Prabal', 'Shetty', 'prabal.shetty@example.com', '1998-11-12'),
('Sneha', 'Shet', 'sneha.shet@example.com', '2002-07-18'),
('Nikhil', 'Gouda', 'nikhil.gouda@example.com', '2002-06-13'),
('Akash', 'Chitragar', 'akash.chitragar@example.com', '2002-12-19'),
('Nithik', 'Joseph', 'nithik.joseph@example.com', '2001-06-05'),
('ADITHYA', 'ACHARYA', 'adithya.acharya@example.com', '1987-01-20'),
('Hardik', 'Prabhu', 'hardik.prabhu@example.com', '2002-04-06'),
('Suchitra', 'Desai', 'suchitra.desai@example.com', '2002-03-08'),
('Akshat', 'Maurya', 'akshat.maurya@example.com', '2002-12-12'),
('Archana', 'Halagannanavar', 'archana.halagannanavar@example.com', '2001-07-22'),
('Deepak', 'Rao', 'deepak.rao@example.com', '1973-11-30'),
('Devi', 'Prasad', 'devi.prasad@example.com', '1969-03-25'),
('ASHOK', 'BONI', 'ashok.boni@example.com', '2000-05-17'),
('Rohan', 'Devang', 'rohan.devang@example.com', '2001-08-10'),
('Imraz', 'Ziya', 'imraz.ziya@example.com', '1994-09-05'),
('PAVANKUMAR', 'NAYAK', 'pavankumar.nayak@example.com', '1997-12-06'),
('Karthik', 'C', 'Karthik.C@example.com','1998-12-15');
""")

# Insert into Accounts
cursor.execute("""
INSERT INTO Accounts (user_id, account_type, balance, currency) VALUES
(1, 'savings', 125000.00, 'INR'),
(1, 'checking', 25000.00, 'INR'),
(2, 'savings', 85000.00, 'INR'),
(3, 'brokerage', 150000.00, 'INR'),
(4, 'savings', 75000.00, 'INR'),
(4, 'checking', 15000.00, 'INR'),
(5, 'brokerage', 250000.00, 'INR'),
(6, 'savings', 45000.00, 'INR'),
(7, 'checking', 95000.00, 'INR'),
(8, 'savings', 110000.00, 'INR'),
(9, 'brokerage', 80000.00, 'INR'),
(10, 'savings', 175000.00, 'INR'),
(11, 'checking', 30000.00, 'INR'),
(12, 'savings', 60000.00, 'INR'),
(13, 'brokerage', 200000.00, 'INR'),
(14, 'savings', 55000.00, 'INR'),
(15, 'checking', 40000.00, 'INR'),
(16, 'savings', 120000.00, 'INR'),
(17, 'brokerage', 90000.00, 'INR'),
(18, 'savings', 70000.00, 'INR'),
(19, 'checking', 35000.00, 'INR'),
(20, 'savings', 105000.00, 'INR');
""")

# Add similar blocks for Transactions, Investments, and Loans here (or let me know if you'd like all that copy-pasted too – it's long)

conn.commit()
cursor.close()
print("Data inserted.")


Data inserted.


In [9]:
cursor = conn.cursor()

# Transactions
cursor.execute("""
INSERT INTO Transactions (account_id, amount, transaction_type, description) VALUES
(1, 15000.00, 'credit', 'Salary deposit'),
(1, 5000.00, 'debit', 'Rent payment'),
(2, 2500.00, 'debit', 'Grocery shopping'),
(3, 10000.00, 'credit', 'Client payment'),
(4, 25000.00, 'credit', 'Investment return'),
(4, 8000.00, 'debit', 'Purchase stocks'),
(5, 3500.00, 'debit', 'Utility bills'),
(6, 4000.00, 'debit', 'Insurance premium'),
(7, 12000.00, 'credit', 'Bonus payment'),
(8, 7500.00, 'credit', 'Tax refund'),
(9, 6000.00, 'debit', 'Online shopping'),
(10, 9000.00, 'credit', 'Interest earned'),
(11, 3000.00, 'debit', 'Restaurant charges'),
(12, 15000.00, 'credit', 'Freelance payment'),
(13, 20000.00, 'credit', 'Dividend income'),
(14, 4500.00, 'debit', 'Medical expenses'),
(15, 8000.00, 'debit', 'Travel booking'),
(16, 12000.00, 'credit', 'Client advance'),
(17, 5500.00, 'debit', 'Electronics purchase'),
(18, 10000.00, 'credit', 'Rental income'),
(19, 3000.00, 'debit', 'School fees'),
(20, 15000.00, 'credit', 'Salary deposit');
""")

# Investments
cursor.execute("""
INSERT INTO Investments (account_id, investment_type, amount, current_value) VALUES
(3, 'stocks', 50000.00, 55000.00),
(4, 'mutual_funds', 75000.00, 80000.00),
(5, 'bonds', 25000.00, 26500.00),
(7, 'crypto', 15000.00, 18000.00),
(9, 'stocks', 30000.00, 32000.00),
(10, 'mutual_funds', 45000.00, 48000.00),
(13, 'crypto', 20000.00, 15000.00),
(13, 'stocks', 80000.00, 90000.00),
(15, 'bonds', 15000.00, 15750.00),
(17, 'mutual_funds', 35000.00, 37500.00),
(18, 'stocks', 20000.00, 22000.00),
(20, 'crypto', 10000.00, 12000.00);
""")

# Loans
cursor.execute("""
INSERT INTO Loans (user_id, loan_amount, interest_rate, loan_term_months, loan_start_date, loan_status) VALUES
(2, 500000.00, 8.50, 60, '2023-01-15', 'active'),
(4, 200000.00, 7.75, 36, '2022-08-20', 'active'),
(5, 1000000.00, 10.25, 120, '2021-12-10', 'active'),
(8, 300000.00, 8.00, 48, '2023-03-05', 'active'),
(9, 150000.00, 7.50, 24, '2022-05-18', 'active'),
(12, 400000.00, 9.75, 60, '2022-11-30', 'active'),
(14, 250000.00, 8.25, 36, '2023-02-12', 'active'),
(16, 750000.00, 10.50, 84, '2021-09-25', 'active'),
(17, 175000.00, 7.25, 24, '2022-07-08', 'closed'),
(19, 350000.00, 9.00, 48, '2023-04-15', 'active');
""")

conn.commit()
cursor.close()
print("Transactions, Investments, and Loans inserted.")


Transactions, Investments, and Loans inserted.


In [10]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    u.user_id,
    CONCAT(u.first_name, ' ', u.last_name) AS full_name,
    u.email,
    SUM(a.balance) AS total_balance
FROM 
    Users u
JOIN 
    Accounts a ON u.user_id = a.user_id
GROUP BY 
    u.user_id, u.first_name, u.last_name, u.email
HAVING 
    total_balance > 100000
ORDER BY 
    total_balance DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

cursor.close()


{'user_id': 5, 'full_name': 'Prabal Shetty', 'email': 'prabal.shetty@example.com', 'total_balance': Decimal('250000.00')}
{'user_id': 13, 'full_name': 'Akshat Maurya', 'email': 'akshat.maurya@example.com', 'total_balance': Decimal('200000.00')}
{'user_id': 10, 'full_name': 'ADITHYA ACHARYA', 'email': 'adithya.acharya@example.com', 'total_balance': Decimal('175000.00')}
{'user_id': 1, 'full_name': 'Sanjay Reddy', 'email': 'sanjay.reddy@example.com', 'total_balance': Decimal('150000.00')}
{'user_id': 3, 'full_name': 'Sakshi Mayya', 'email': 'sakshi.mayya@example.com', 'total_balance': Decimal('150000.00')}
{'user_id': 16, 'full_name': 'Devi Prasad', 'email': 'devi.prasad@example.com', 'total_balance': Decimal('120000.00')}
{'user_id': 8, 'full_name': 'Akash Chitragar', 'email': 'akash.chitragar@example.com', 'total_balance': Decimal('110000.00')}
{'user_id': 20, 'full_name': 'PAVANKUMAR NAYAK', 'email': 'pavankumar.nayak@example.com', 'total_balance': Decimal('105000.00')}


True

In [ ]:
cursor = conn.cursor()

indexes = [
    "CREATE INDEX idx_accounts_user_id ON Accounts(user_id);",
    "CREATE INDEX idx_balance ON Accounts(balance);",
    "CREATE INDEX idx_users_name_email ON Users(first_name, last_name, email);",
    "CREATE INDEX idx_account_type ON Accounts(account_type);",
    "CREATE INDEX idx_account_id_transactions ON Transactions(account_id);",
    "CREATE INDEX idx_transaction_date ON Transactions(transaction_date);",
    "CREATE INDEX idx_account_id_investments ON Investments(account_id);",
    "CREATE INDEX idx_investment_type ON Investments(investment_type);",
    "CREATE INDEX idx_user_id_loans ON Loans(user_id);",
    "CREATE INDEX idx_loan_status ON Loans(loan_status);"
]

for index_query in indexes:
    try:
        cursor.execute(index_query)
    except mysql.connector.Error as e:
        print(f"Skipping (maybe exists): {e}")

conn.commit()
cursor.close()
print("Indexes added.")

Skipping (maybe exists): 1061 (42000): Duplicate key name 'idx_account_type'
Skipping (maybe exists): 1061 (42000): Duplicate key name 'idx_transaction_date'
Skipping (maybe exists): 1061 (42000): Duplicate key name 'idx_investment_type'
Skipping (maybe exists): 1061 (42000): Duplicate key name 'idx_loan_status'
Indexes added.


In [12]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    account_type, 
    SUM(balance) AS total_balance
FROM 
    Accounts USE INDEX (idx_accounts_user_id, idx_account_type)
WHERE 
    user_id = 1
GROUP BY 
    account_type;
"""

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

cursor.close()


{'account_type': 'savings', 'total_balance': Decimal('125000.00')}
{'account_type': 'checking', 'total_balance': Decimal('25000.00')}


True

In [13]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    t.transaction_id, t.amount, t.transaction_type, t.transaction_date
FROM 
    Transactions t USE INDEX (idx_account_id_transactions, idx_transaction_date)
WHERE 
    t.account_id = 1
    AND t.transaction_date >= CURDATE() - INTERVAL 30 DAY
ORDER BY 
    t.transaction_date DESC;
"""

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

cursor.close()


{'transaction_id': 1, 'amount': Decimal('15000.00'), 'transaction_type': 'credit', 'transaction_date': datetime.datetime(2025, 4, 20, 18, 52, 17)}
{'transaction_id': 2, 'amount': Decimal('5000.00'), 'transaction_type': 'debit', 'transaction_date': datetime.datetime(2025, 4, 20, 18, 52, 17)}


True

In [14]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    investment_type, amount, current_value, purchase_date
FROM 
    Investments USE INDEX (idx_account_id_investments, idx_investment_type)
WHERE 
    account_id = 13
    AND investment_type = 'stocks';
"""

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

cursor.close()

{'investment_type': 'stocks', 'amount': Decimal('80000.00'), 'current_value': Decimal('90000.00'), 'purchase_date': datetime.datetime(2025, 4, 20, 18, 52, 17)}


True

In [15]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    loan_amount, interest_rate, loan_term_months, loan_start_date
FROM 
    Loans USE INDEX (idx_user_id_loans, idx_loan_status)
WHERE 
    user_id = 5
    AND loan_status = 'active';
"""

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

cursor.close()


{'loan_amount': Decimal('1000000.00'), 'interest_rate': Decimal('10.25'), 'loan_term_months': 120, 'loan_start_date': datetime.date(2021, 12, 10)}


True

In [16]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    user_id, first_name, last_name, email
FROM 
    Users USE INDEX (idx_users_name_email)
WHERE 
    first_name LIKE 'S%'
    AND email LIKE '%example.com';
"""

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

cursor.close()


{'user_id': 3, 'first_name': 'Sakshi', 'last_name': 'Mayya', 'email': 'sakshi.mayya@example.com'}
{'user_id': 1, 'first_name': 'Sanjay', 'last_name': 'Reddy', 'email': 'sanjay.reddy@example.com'}
{'user_id': 6, 'first_name': 'Sneha', 'last_name': 'Shet', 'email': 'sneha.shet@example.com'}
{'user_id': 12, 'first_name': 'Suchitra', 'last_name': 'Desai', 'email': 'suchitra.desai@example.com'}


True

In [17]:
cursor = conn.cursor(dictionary=True)

query = """
SELECT 
    account_id, user_id, account_type, balance
FROM 
    Accounts USE INDEX (idx_balance)
WHERE 
    balance > 100000
ORDER BY 
    balance DESC;
"""

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

cursor.close()


{'account_id': 7, 'user_id': 5, 'account_type': 'brokerage', 'balance': Decimal('250000.00')}
{'account_id': 15, 'user_id': 13, 'account_type': 'brokerage', 'balance': Decimal('200000.00')}
{'account_id': 12, 'user_id': 10, 'account_type': 'savings', 'balance': Decimal('175000.00')}
{'account_id': 4, 'user_id': 3, 'account_type': 'brokerage', 'balance': Decimal('150000.00')}
{'account_id': 1, 'user_id': 1, 'account_type': 'savings', 'balance': Decimal('125000.00')}
{'account_id': 18, 'user_id': 16, 'account_type': 'savings', 'balance': Decimal('120000.00')}
{'account_id': 10, 'user_id': 8, 'account_type': 'savings', 'balance': Decimal('110000.00')}
{'account_id': 22, 'user_id': 20, 'account_type': 'savings', 'balance': Decimal('105000.00')}


True